In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
import numpy as np
import joblib

In [ ]:
train_image_path = 'dataset/train_images.idx3-ubyte'
train_label_path = 'dataset/train_labels.idx1-ubyte'
test_image_path  = 'dataset/test_image.idx3-ubyte'
test_label_path  = 'dataset/test_label.idx1-ubyte'

In [ ]:
def image_process(path_file):
    with open(path_file, 'rb') as file:
        data = np.fromfile(file, np.uint8, offset=16)
        data = data / 255
        return data.reshape(-1, 28, 28)


In [ ]:
def label_process(path_file):
    with open(path_file, 'rb') as file:
        data = np.fromfile(file, np.uint8, offset=8)
        return data


In [ ]:
X_train = image_process(train_image_path)
X_test  = image_process(test_image_path)
y_train = label_process(train_label_path)
y_test  = label_process(test_label_path)

In [ ]:
X_train.shape , y_train.shape , X_test.shape , y_test.shape

In [ ]:
fig = plt.figure(figsize=(20,10))
for i in range(10):
    ax = plt.subplot(1,13,i+1)
    ax.imshow(X_train[i] , cmap='gray')
    plt.title('Label ' + str(y_train[i]))
    plt.xticks([])
    plt.yticks([])
    plt.show

In [ ]:
# # validation set 
# random_indices = np.random.permutation(X_train.shape[0])
# length_i = int(X_train.shape[0]*0.2)  # 20% pour la validation
# val_indices = random_indices[: length_i]
# train_indices = random_indices[length_i :]
# X_val = X_train[val_indices]
# X_train = X_train[train_indices]
# y_val = y_train[val_indices]
# y_train = y_train[train_indices]

In [ ]:
# X_train.shape , y_train.shape , X_val.shape , y_val.shape 

# Modele 1 : K-nearest neighbors

In [ ]:
class KNNClassifier:
    def euclideanDistance(self, x1, x2):
        return np.sqrt(np.sum(np.square(x1 - x2))) 
    
    def manhattanDistance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predect(self, k, X_test, distance='euclidean', weights='uniform'):
        y_pred = []
        
        for x in X_test:
            if distance == 'manhattan':
                distances = [self.manhattanDistance(x, x_train) for x_train in self.X_train]
            else:
                distances = [self.euclideanDistance(x, x_train) for x_train in self.X_train]
                     
            k_indices = np.argsort(distances)[:k]
            k_labels = self.y_train[k_indices]
            
            if weights == 'distance':
                y = k_labels[np.argmax(np.bincount(k_labels) * (1 / distances[k_indices]))]
                y_pred.append(y)
            else:
                y = np.argmax(np.bincount(k_labels))
                y_pred.append(y)
        
        return np.array(y_pred)    

In [ ]:
# PARAMETRES 
k = 4
distance_type = 'manhattan'
weight_type = 'uniform'

# prend en petit partie du data

xpetit_train = X_train[ : 500]
ypetit_train = y_train[ : 500]

xpetit_test = X_test[ : 10]
ypetit_test = y_test[ : 10]

xpetit_train = np.reshape(xpetit_train, (xpetit_train.shape[0], -1))
train = np.reshape(ypetit_train, (ypetit_train.shape[0], -1))
xpetit_test = np.reshape(xpetit_test, (xpetit_test.shape[0], -1))
ypetit_test = np.reshape(ypetit_test, (ypetit_test.shape[0], -1))



In [ ]:
model = KNNClassifier()

model.fit(xpetit_train , ypetit_train)

ypred = model.predect( k , xpetit_test, distance=distance_type, weights=weight_type)

accuracy = accuracy_score(ypetit_test, ypred)
print("Accuracy : {:.2f}%".format(accuracy * 100))

In [ ]:
model = KNeighborsClassifier(n_neighbors=k)

model.fit(xpetit_train, ypetit_train)

y_pred = model.predict(xpetit_test)

accuracy = accuracy_score(ypetit_test, y_pred)
print("Accuracy : {:.2f}%".format(accuracy * 100))

# Model 2 : Logistique regression

In [ ]:
class LogistiqueClssifieur:
    
    def __init__(self, alpha, iterations, normalize = True):
        self.alpha = alpha
        self.iterations = iterations    
        self.normalize = normalize    
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X_train, y_train):
        # recuperer nombre du classe 
        self.classes = np.unique(y_train)
        self.nombre_classes = len(self.classes)
        
        # pour ajouter 1 
        intercept = np.ones((X_train.shape[0], 1))
        self.X_train = X_train.reshape(X_train.shape[0] , -1)
        
        # faire normalisation
        if self.normalize :
            scaler = StandardScaler()
            scaler.fit( self.X_train)
            self.X_train = np.column_stack((intercept, scaler.transform(self.X_train)))
        else : self.X_train = np.column_stack((intercept, X_train))
        
        self.y_train = y_train.reshape(y_train.shape[0] , 1)  
        #initilaiser les thetas      
        self.theta = np.zeros((self.nombre_classes , self.X_train.shape[1]))
        # mise a jour les theta 
        self.predictMultiClasses()
            
    def derivative(self, theta , y):       
        m = len(self.X_train)
        h_theta = self.sigmoid(self.X_train @ theta.T)
        return (self.X_train.T @ (h_theta - y)).T / m    
    
    def computeCostLogistique(self, theta , y):
        m = len(self.X_train)
        h_theta = self.sigmoid(self.X_train @ theta.T )        
        h_theta[h_theta == 0.] += np.finfo(float).eps
        h_theta[h_theta == 1.] -= np.finfo(float).eps  
        return - np.sum( y.T @ np.log(h_theta) + (1 - y).T @ np.log(1 - h_theta))  / m

    def gradientDescent(self , y):
        theta_optimum = np.zeros((1 , self.X_train.shape[1]))
        theta = np.zeros((1 , self.X_train.shape[1]))
        cost_optimum = self.computeCostLogistique(theta,y)
        for i in range(self.iterations):
            delta_theta = self.derivative(theta , y)            
            theta = theta - self.alpha * delta_theta         
            cost = self.computeCostLogistique(theta, y)
            if cost_optimum > cost:
                theta_optimum = np.array(theta)
                cost_optimum = cost
        return theta_optimum
    
    def predictMultiClasses(self ):                
        for i in range(self.nombre_classes):            
            theta = self.gradientDescent( ( self.y_train == self.classes[i] ).astype(int))
            self.theta[i, :] = theta       
    
    def predict(self, X_test):
        intercept = np.ones((X_test.shape[0], 1))
        X_test = X_test.reshape(X_test.shape[0] , -1)
        if self.normalize :
            scaler = StandardScaler()
            scaler.fit(X_test)
            X_test = np.column_stack((intercept, scaler.transform(X_test)))
        else :  X_test = np.column_stack((intercept, X_test) ) 
        proba = X_test @ self.theta.T
        max_proba_classe = np.argmax(proba , axis=1)
        predect = self.classes[ max_proba_classe]
        return predect.reshape((-1, 1))

## Train and Save the model

In [ ]:
#model = LogistiqueClssifieur(0.01 , 1000 , False)
#model.fit(xpetit_train, ypetit_train)
# save modele 
filename = 'LogistiqueClassifieurModel.sav'
#joblib.dump(model, filename)

In [ ]:
# open model
model = joblib.load(filename)

In [ ]:
ypred = model.predict(xpetit_test)
ypetit_test = ypetit_test.reshape(xpetit_test.shape[0] , 1)
accuracy = accuracy_score(ypetit_test, ypred)
print("Accuracy : {:.2f}%".format(accuracy * 100))

# Model 3 : RNN

In [ ]:
class RNN:
    def __init__(self , hidden_layer_sizes, alpha, iterations , normalize = True) :
        self.hidden_layer_sizes = hidden_layer_sizes
        self.alpha = alpha
        self.iterations = iterations
        self.normalize = normalize
        
    def initialization_weights(self):
        low = - 0.1
        high = 0.1 
        self.weights = []
        self.baiais  = []       
        # Layer 1  
        w = np.random.uniform(low, high, size = (self.hidden_layer_sizes[0], self.input_layer_size ))
        self.weights.append(w)
        b = np.random.uniform(low, high, size = (w.shape[0] , 1))  
        self.baiais.append(b)   
                
        for i in range((len(self.hidden_layer_sizes)) - 1 ) :
            w = np.random.uniform(low, high, size = (self.hidden_layer_sizes[i+1], self.hidden_layer_sizes[i]))
            self.weights.append(w)
            b = np.random.uniform(low, high, (w.shape[0] , 1))  
            self.baiais.append(b)   
             
        #  outout layer
        w = np.random.uniform(low, high, size = ( self.output_layer_size , self.hidden_layer_sizes[-1]) )
        b = np.random.uniform(low, high, (w.shape[0] , 1))   
        self.baiais.append(b)    
        self.weights.append(w)         
        return self.weights , self.baiais
    
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    
    def activation( z):
        return RNN.sigmoid(z)
    
    def lossFunction(self, weights, baiais):
        m = self.y_train.shape[0]        
        a = RNN.forwardPropagation(self.X_train , weights, baiais)
        return - np.sum(  self.YY @ np.log(a[-1]) +  ( 1 - self.YY ) @ np.log( 1 - a[-1] ) ) / m 
     
    def forwardPropagation(X, weights, baiais):
        a_all_layer = []
        a = np.transpose(X)
        a_all_layer.append(a)
        
        for i in range(len(weights)): 
            z = weights[i] @ a + baiais[i]
            a = RNN.activation(z)   
            a_all_layer.append(a)
            
        return a_all_layer
            
    def backPropagation(self):
        weights , baises = self.initialization_weights()
        cost_optimum = self.lossFunction(weights , baises)
                
        for i in range(self.iterations):
            a = RNN.forwardPropagation(self.X_train , weights , baises)  # 4 ACTIVATIONS

            dalta_weights = []
            dalta_baises = []
            
            # dz output layer
            dz = a[-1] - self.YY.T    # 10 * 500 
            dw = dz @ a[-2].T ###########################################

            dalta_weights.append(dw)
            dalta_baises.append(dz) # db = dz
            
            for L  in range( len(self.hidden_layer_sizes) - 1 , -1 , -1  ):             
                dz = weights[ L + 1].T @ dz * a[L + 1] * ( 1 - a[L + 1])
                
                dw = dz @ a[L].T ##################
                
                dalta_weights = [ dw ] + dalta_weights
                dalta_baises = [ dz ] + dalta_baises            
            
            
            mean_dalta_weights = [np.mean(dalta_weight , axis = 0).reshape((-1,1)) for dalta_weight in dalta_weights]
            mean_dalta_baises = [np.mean(dalta_baises , axis = 1).reshape((-1,1))  for dalta_baises in dalta_baises]
            
            for k in range(len(weights)):
               
                weights[k] = weights[k] - self.alpha * mean_dalta_weights[k].T
                baises[k] = baises[k] - self.alpha * mean_dalta_baises[k]  

            cost = self.lossFunction(weights, baises)
            if cost < cost_optimum : 
                cost_optimum = cost
                self.weights = weights
                self.baiais = baises   
        print(cost_optimum)
        return self.weights , self.baiais
        
    def fit(self, X_train , y_train):
        # recuperer nombre du classe 
        self.classes = np.unique(y_train)
        self.nombre_classes = len(self.classes)
        
        self.X_train = X_train
        self.y_train = y_train
        
        self.YY = np.zeros((y_train.shape[0] , self.nombre_classes ))
        
        for i in range(self.nombre_classes):
            self.YY[ : , i ] = ( self.y_train == self.classes[i] ).astype(int)
            
        # faire normalisation
        if self.normalize :
            scaler = StandardScaler()
            scaler.fit( self.X_train)
            self.X_train = scaler.transform(self.X_train)

        self.input_layer_size =  X_train.shape[1]
        self.output_layer_size = self.nombre_classes
        
        self.backPropagation()
        
        return -1
        
    def predict(self, X_test ):       
        proba = RNN.forwardPropagation(X_test , self.weights, self.baiais)[-1]
        max_proba_classe = np.argmax(proba , axis=0)
        predect = self.classes[ max_proba_classe]
        return predect.reshape((-1, 1))

In [ ]:
model = RNN( (15 , ) , 0.01 , 1000)
model.fit(xpetit_train , ypetit_train)


In [ ]:
ypred = model.predict(xpetit_test)
accuracy = accuracy_score(ypetit_test, ypred)
print("Accuracy : {:.2f}%".format(accuracy * 100))

In [ ]:
class NeuralNet:
    def __init__(self , hidden_layer):
        self.hidden_layer = hidden_layer
        self._fix_parameters()   
        self.activation = NeuralNet.sigmoid    
    
    def _fix_parameters(self , lr = 0.01 , epoches = 1000 , normalize = True):
        self.lr = lr 
        self.epoches = epoches 
        self.normalize = normalize 
        
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    
    def forward(self , X, weights , baias):        
        a_all_layer = []
        a = X.T     
        a_all_layer.append(a)   
        for i in range(len(weights)):
            z = weights[i] @ a + baias[i]
            a = self.activation(z)
            a_all_layer.append(a)            
        return a_all_layer
       
    def lossFunction(self , X   , YY , weights , baias):
        ypred = self.forward(X , weights , baias) [-1]
        return np.mean(YY @ np.log(ypred) )
         
       
    def initialization_weights(self):
        self.weights = []
        self.baias = [] 
        
        low = - 0.1
        high = 0.1
        
        # input layer 
        w = np.random.uniform(low, high, size = (self.hidden_layer[0], self.input_layer_size ))
        self.weights.append(w)
        b = np.random.uniform(low, high, size = (w.shape[0] , 1))  
        self.baias.append(b)   
                
        for i in range((len(self.hidden_layer)) - 1 ) :
            w = np.random.uniform(low, high, size = (self.hidden_layer[i+1], self.hidden_layer[i]))
            self.weights.append(w)
            b = np.random.uniform(low, high, (w.shape[0] , 1))  
            self.baias.append(b)   
             
        #  outout layer
        w = np.random.uniform(low, high, size = ( self.output_layer_size , self.hidden_layer[-1]) )
        b = np.random.uniform(low, high, (w.shape[0] , 1))   
        self.baias.append(b)    
        self.weights.append(w)   
              
        return self.weights , self.baias
    
    def backward(self , a_all , weights ):
        
        dalta_weights = []
        dalta_baises = []
            
        # dz output layer
        dz = a_all[-1] - self.YY.T    # 10 * 500 
        dw = dz @ a_all[-2].T ###########################################

        dalta_weights.append(dw)
        dalta_baises.append(dz) # db = dz
            
        for L  in range( len(self.hidden_layer) - 1 , -1 , -1  ):             
            dz = weights[ L + 1].T @ dz * a_all[L + 1] * ( 1 - a_all[L + 1])
                
            dw = dz @ a_all[L].T ##################
              
            dalta_weights = [ dw ] + dalta_weights
            dalta_baises = [ dz ] + dalta_baises            
            
            
            mean_dalta_weights = [np.mean(dalta_weight , axis = 0).reshape((-1,1)) for dalta_weight in dalta_weights]
            mean_dalta_baises = [np.mean(dalta_baises , axis = 1).reshape((-1,1))  for dalta_baises in dalta_baises] 
            
        return mean_dalta_weights , mean_dalta_baises
               
        
    def fit(self, X_train , y_train):
        
        self.classes = np.unique(y_train)
        self.nombre_classes = len(self.classes)
        
        self.X_train = X_train
        self.y_train = y_train
        
       
            
            
        # faire normalisation
        if self.normalize :
            scaler = StandardScaler()
            scaler.fit( self.X_train)
            self.X_train = scaler.transform(self.X_train)

        self.input_layer_size =  X_train.shape[1]
        self.output_layer_size = self.nombre_classes
        
        weights , baias = self.initialization_weights()
        loss_optim = self.lossFunction(self.X_train , self.YY , self.weights , self.baias)
        
        for epoch in range(self.epoches):
            
            a = self.forward(self.X_train , weights , baias)
        
            dw , db  = self.backward( a , weights)

            
            for i in range(len(weights)):
               
                weights[i] = weights[i] - self.lr * dw[i].T
                baias[i] = baias[i] - self.lr * db[i]  

            loss = self.lossFunction(self.X_train , self.YY , weights, baias)
            if loss < loss_optim : 
                loss_optim = loss
                self.weights = weights
                self.baiais = baias   
                
        print(loss_optim)
        
        return self.weights , self.baias
    
    def predict(self, X_test ):       
        proba = RNN.forwardPropagation(X_test , self.weights, self.baiais)[-1]
        print(proba)
        max_proba_classe = np.argmax(proba , axis=0)
        predect = self.classes[ max_proba_classe]
        return predect.reshape((-1, 1))
        
        
        
        
    

In [ ]:
model = NeuralNet((15 ,))
model.fit(xpetit_train , ypetit_train)


In [ ]:
ypred = model.predict(xpetit_train)
accuracy = accuracy_score(ypetit_train, ypred)
print("Accuracy : {:.2f}%".format(accuracy * 100))

In [ ]:
def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    

In [51]:
w1 = np.random.uniform( -0.1 , 0.1 , size = ( 15 , 784))
b1 = np.random.uniform( -0.1, 0.1 , size=(15 , 1))
w2 = np.random.uniform( -0.1 , 0.1 , size = ( 10 , 15))
b2 = np.random.uniform( -0.1, 0.1 , size=(10 , 1))

def forward(X):
    a1 = sigmoid(w1 @ X.T)
    a2 = sigmoid(w2 @ a1 )
    return a1 , a2 

def backward(X , a1 , a2):
   
   dz2 = a2 - YY.T # (10 * 500)
   dw2 = dz2 @ a1.T    # 10 * 500 @ 500 * 15   =  10 * 15           
   db2 = dz2       # 10 * 500
   
   dz1 = w2.T @ dz2 * a1 * ( 1 - a1 )   
   dw1 = dz1 @ X
   db2 = dz1 
                
   db2 = np.mean(db2 , axis = 1)
   db1 = np.mean(db1 , adix = 1)
   
   print(db1.shape , db2.shape)
   
    
   return dw1 , db1 , dw2 , db2 
               
        

def lossFunction(X , YY):
    _, ypred = forward(X)
    ypred = ypred.T
    return - np.mean( YY * np.log(ypred))
    
YY = np.zeros(( ypetit_train.shape[0] , 10 )   )  
for i in range(10):
    YY[ : , i ] = ( ypetit_train == i ).astype(int)
    
    
num_epochs = 100
learning_rate = 0.001

optimale = ( w1 , b1 , w2 , b2)
lossOptim = lossFunction(xpetit_train)

def gradinetDescent(X):
    
    for epoch in range(num_epochs):
        
        a1 , a2 = forward(xpetit_train)
        
    
    
    
    


0.07224444086086405

In [44]:
a1 , a2 = forward(xpetit_train)
a2.shape

(10, 500)